In [6]:
import sqlite3
import pandas as pd
import io

In [7]:
conn = sqlite3.connect('//kaggle//input//subtitle//eng_subtitles_database.db')
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
print(cursor.fetchall())

[('zipfiles',)]


In [8]:
cursor.execute("PRAGMA table_info('zipfiles')")
cols = cursor.fetchall()
for col in cols:
    print(col[1])

num
name
content


In [9]:
df = pd.read_sql_query("""SELECT * FROM zipfiles""", conn)
df.head()

,num,name,content
0,9180533,the.message.(1976).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x1c\xa9\x...
1,9180583,here.comes.the.grump.s01.e09.joltin.jack.in.bo...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x17\xb9\x...
2,9180592,yumis.cells.s02.e13.episode.2.13.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00L\xb9\x99V...
3,9180594,yumis.cells.s02.e14.episode.2.14.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00U\xa9\x99V...
4,9180600,broker.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x001\xa9\x99V...


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82498 entries, 0 to 82497
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   num      82498 non-null  int64 
 1   name     82498 non-null  object
 2   content  82498 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.9+ MB


In [11]:
import zipfile
import regex as re

In [12]:
def decode(binary_data_filelike):
    with zipfile.ZipFile(binary_data_filelike, 'r') as zip_ref:
        extracted_file_info = zip_ref.infolist()[0]  # assuming only one file is there
        extracted_file_name = extracted_file_info.filename
        with zip_ref.open(extracted_file_name) as extracted_file:
            decoded_text = extracted_file.read().decode('latin-1')
    decoded_text = decoded_text.replace('\n','')
    decoded_text = re.sub('<[^>]*>|[^a-zA-Z\s.\']','',decoded_text)
    decoded_text = decoded_text.split('\r')
    for i in decoded_text:
        if len(i)<=2:
            decoded_text.remove(i)
    return decoded_text

In [13]:
df['dec_cont'] = df['content'][:2500].apply(lambda x : decode(io.BytesIO(x)))

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82498 entries, 0 to 82497
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   num       82498 non-null  int64 
 1   name      82498 non-null  object
 2   content   82498 non-null  object
 3   dec_cont  2500 non-null   object
dtypes: int64(1), object(3)
memory usage: 2.5+ MB


In [15]:
new_df = df.head(2500)
new_df.head()

,num,name,content,dec_cont
0,9180533,the.message.(1976).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x1c\xa9\x...,"[Watch any video online with OpenSUBTITLES, Fr..."
1,9180583,here.comes.the.grump.s01.e09.joltin.jack.in.bo...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x17\xb9\x...,"[Ah There's Princess, Dawn and Terry with the,..."
2,9180592,yumis.cells.s02.e13.episode.2.13.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00L\xb9\x99V...,"[Yumi's Cells , Episode , Extremely Polite Yum..."
3,9180594,yumis.cells.s02.e14.episode.2.14.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00U\xa9\x99V...,"[Watch any video online with OpenSUBTITLES, Fr..."
4,9180600,broker.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x001\xa9\x99V...,"[Watch any video online with OpenSUBTITLES, Fr..."


In [16]:
def chunkie(x):
    li=[]
    for i in range(0,len(x),10):
        li.append(x[i:i+10])
    if(len(x)%10!=0):
        li.append(x[(len(x)//10)*10:])
    return li

In [17]:
new_df['dec_cont'] = new_df['dec_cont'].apply(lambda x: chunkie(x))

/tmp/ipykernel_33/3751119736.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['dec_cont'] = new_df['dec_cont'].apply(lambda x: chunkie(x))


In [18]:
new_df.drop('content',axis=1,inplace=True)

/tmp/ipykernel_33/1046226730.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.drop('content',axis=1,inplace=True)


In [19]:
chunk_df = new_df.explode('dec_cont')

In [20]:
chunk_df.info()
chunk_df.index = [i for i in range(501903)]

<class 'pandas.core.frame.DataFrame'>
Index: 501903 entries, 0 to 2499
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   num       501903 non-null  int64 
 1   name      501903 non-null  object
 2   dec_cont  501903 non-null  object
dtypes: int64(1), object(2)
memory usage: 15.3+ MB


In [21]:
def list2text(x):
    txt=""
    for i in x:
        txt = txt + i
    return txt
chunk_df['dec_cont'] = chunk_df['dec_cont'].apply(lambda x : list2text(x))

In [22]:
chunk_df.head()

,num,name,dec_cont
0,9180533,the.message.(1976).eng.1cd,Watch any video online with OpenSUBTITLESFree ...
1,9180533,the.message.(1976).eng.1cd,He speaks of a new prophet in Arabia.Was it li...
2,9180533,the.message.(1976).eng.1cd,smelling of camel and goat.To tell Persia wher...
3,9180533,the.message.(1976).eng.1cd,which holds that the Impersonation of the Prop...
4,9180533,the.message.(1976).eng.1cd,Mecca was then a rich tradingcity ruled by its...


In [1]:
pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.1 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of opentelemetry-sdk to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 10.1 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 37.4 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 698.9/698.9 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 44.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 66.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41

In [24]:
import chromadb
from chromadb.config import Settings


client = chromadb.PersistentClient(path="db")

In [25]:
collection = client.create_collection(name="Students")

In [26]:
documents = []
metadatas = []
ids = []

for index, row in chunk_df.iterrows():
    documents.append(row['dec_cont'])
    metadata = {"chapter": row['name'], "verse": row['dec_cont']}
    metadatas.append(metadata)
    ids.append(f"id{index}")


In [27]:
collection.add(documents=documents[:1000], metadatas=metadatas[:1000], ids=ids[:1000])

/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:07<00:00, 10.6MiB/s]


In [28]:
results = collection.query(
    query_texts=["You think darkness is your ally"],
    n_results=4
)

In [29]:
results

{'ids': [['id280', 'id684', 'id176', 'id562']],
 'distances': [[1.1283966302871704,
   1.2388193607330322,
   1.2830294370651245,
   1.3151577711105347]],
 'metadatas': [[{'chapter': 'the.message.(1976).eng.1cd',
    'verse': 'Bedouins broke the truce not usThe night was darkI am here to testifythe night was dark'},
   {'chapter': 'yumis.cells.s02.e14.episode.2.14.(2022).eng.1cd',
    'verse': ' I spend the day constantly acting fearless and before I know it the night arrives  The dark night '},
   {'chapter': 'the.message.(1976).eng.1cd',
    'verse': 'BATTLE OF BADRWho are youWe expect our peersand our equals'},
   {'chapter': 'yumis.cells.s02.e14.episode.2.14.(2022).eng.1cd',
    'verse': " I will become the moon just for you  So that you can breathe within me  Darkness and loneliness entirely  will cease to exist  Cause you're my star  No matter what kind of night falls I will shine on you  so that you can fall asleep in the warmth of the moonlight  Cause you are my star  Cause you

In [30]:
result1 = collection.query(
    query_texts = ['Destroyer of worlds'],n_results=1
)

In [31]:
result1

{'ids': [['id257']],
 'distances': [[1.2723112106323242]],
 'metadatas': [[{'chapter': 'the.message.(1976).eng.1cd',
    'verse': '...call the world to IslamTo Heraclius Emperor of ByzantiumKisra Emperor of Persia'}]],
 'embeddings': None,
 'documents': [['...call the world to IslamTo Heraclius Emperor of ByzantiumKisra Emperor of Persia']],
 'uris': None,
 'data': None}

In [32]:
!zip -r 'db.zip' 'db'

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  adding: db/ (stored 0%)
  adding: db/75ba601b-c84e-4294-83f9-5cc954901c5f/ (stored 0%)
  adding: db/75ba601b-c84e-4294-83f9-5cc954901c5f/index_metadata.pickle (deflated 52%)
  adding: db/75ba601b-c84e-4294-83f9-5cc954901c5f/header.bin (deflated 56%)
  adding: db/75ba601b-c84e-4294-83f9-5cc954901c5f/data_level0.bin (deflated 12%)
  adding: db/75ba601b-c84e-4294-83f9-5cc954901c5f/length.bin (deflated 96%)
  adding: db/75ba601b-c84e-4294-83f9-5cc954901c5f/link_lists.bin (deflated 86%)
  adding: db/chroma.sqlite3 (deflated 49%)


In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [30]:
tfidf = TfidfVectorizer()

In [31]:
result = tfidf.fit_transform(chunk_df['dec_cont'].to_list())

In [1]:
doc = []
embed = []
metadatas = []
ids = []
for index, row in chunk_df.iterrows():
    doc.append(row['dec_cont'])
    metadata = {"chapter": row['name'], "verse": row['dec_cont']}
    metadatas.append(metadata)
    tfidf_matrix = tfidf.fit_transform([row['dec_cont']])
    embe = tfidf_matrix.toarray().tolist()
    embed.append(embe)

    ids.append(f"id{index}")

NameError: name 'chunk_df' is not defined

In [77]:
result.shape

(49800, 83437)

In [33]:
import zipfile
import os

with zipfile.ZipFile('db.zip', 'r') as zip_ref:
    zip_ref.extractall('db')


In [2]:
import chromadb
from chromadb.config import Settings

client = chromadb.PersistentClient(path="/kaggle/input/xgrdcguhkijl/db")
print(client.list_collections())
collection = client.get_collection(name="Students")

[Collection(name=Students)]


In [3]:
results = collection.query(
    query_texts=["Forest for the trees.Detective"],
    n_results=10
)

/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:01<00:00, 45.2MiB/s]


In [4]:
results

{'ids': [['id8081',
   'id9443',
   'id8973',
   'id4849',
   'id882',
   'id9659',
   'id6366',
   'id4310',
   'id4717',
   'id7378']],
 'distances': [[1.1080106496810913,
   1.2120002508163452,
   1.212144374847412,
   1.2123314142227173,
   1.2367467880249023,
   1.2499502897262573,
   1.2566027641296387,
   1.261085033416748,
   1.2640762329101562,
   1.270662546157837]],
 'metadatas': [[{'chapter': 'the.governor.s02.e02.episode.2.2.(1996).eng.1cd',
    'verse': 'Cheers.Plays Willow Titwillow On a tree by a rivera little tomtit '},
   {'chapter': 'the.governor.s01.e01.episode.1.1.(1995).eng.1cd',
    'verse': "Royston Andrews speaking.It's Marshall.I think you should knowthe Winchwood familyhave turned up here.Just find out whatthe Winchwoods wantand then get back to meas soon as you can.No."},
   {'chapter': 'the.governor.s02.e05.episode.2.5.(1996).eng.1cd',
    'verse': "Not in your plan.Hey Officer Chiswickwant to tell the noncesyou can't knock off the tree"},
   {'chapter': 't